In [9]:
from TrajGen import (
    TrajectoryGenerator,
    DataPoint,
    TrajectoryGeneratorCollection,
    WayPoint,
    color_hex,
    rand_24_bit,
    plot_trajectory,
    plot_datapoint,
    setup_map,
)
import os
import gpxpy
import folium
import random
from typing import Tuple, List
from folium import plugins
import matplotlib.pyplot as plt
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import uuid
import utm 
import gpxpy
import json
from shapely.geometry import Point, Polygon, LineString
from shapely import ops

center = [55.39594, 10.38831]

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

In [3]:
kmh_to_ms = (1000 / (60 * 60))
# Set trajectory charateristics
mean_speed = 4.0 * kmh_to_ms  # Average human walking speed in m/s
std_speed = 0.5 * kmh_to_ms   # Deviation in walking speed in m/s
mean_time_delta = 5.
std_time_delta = 20.

def ncrands(n: int, min_: int, max_: int):
    rs = []; r0 = np.random.randint(min_, max_)
    for _ in range(n):
        r1 = np.random.randint(min_, max_ - 1) 
        r2 = (r0 + r1 + 1) % max_
        r0 = r2
        rs.append(r2)
    return rs

def generate_pattern(profile):
    is_anom = profile[0]
    n_patterns = profile[1]
    pattern = profile[2]
    start_pair = WayPoint(
        latitude = pattern[0][0],
        longitude = pattern[0][1],
        duration = 60, # In seconds
        std = 0, # In meters
    )
    end_pair = WayPoint(
        latitude = pattern[-1][0],
        longitude = pattern[-1][1],
        duration = 60, # In seconds
        std = 0, # In meters
    )
    middle_pairs = []
    for i in range(1, len(pattern) - 1):
        middle_pairs.append(
            WayPoint(
                latitude = pattern[i][0],
                longitude = pattern[i][1],
                duration = 0, # Seconds
                std = 0, # In meters
            )
        )
    list_waypoints = [(is_anom, n_patterns, 
            [start_pair] + middle_pairs + [end_pair]
        )
    ]
    return list_waypoints

In [220]:
locations = {    
    "home": [55.367946094149396, 10.405759610262596],
    "fakta": [55.364467731441266, 10.392707986720698],
    "lidl": [55.37274058513472, 10.40862169924273],
    "land": [55.36595581773891, 10.395365460132705],
    "hcl": [55.37298467679492, 10.39972562981267],
    "uni": [55.368209519523774, 10.42749124096433],
    "netto": [55.37495502665289, 10.427400972127419],
}

profiles = {
    #### home --> fakta (5 + 2). fakta --> home (3 + ?)
    "home-fakta-norm": [False, 6, [locations["home"], locations["fakta"]]],
    "fakta-home-norm": [False, 7 + 7,
        [
            locations["fakta"],
            [55.36444916499685, 10.405136107684182],
            locations["home"],
        ]
    ],
    "home-fakta-devi1": [True, 1,
        [
            locations["home"],
            [55.3688962638592, 10.3985793181517],
            [55.3653719858155, 10.398257453074862],
            locations["fakta"],
        ]
    ],
    "home-fakta-devi2": [True, 1,
        [
            locations["home"],
            [55.3687011563267, 10.401390273156068],
            [55.36571345209057, 10.401175696438177],
            locations["fakta"],
        ]
    ],
    ##
    "fakta-home-devi1": [True, 1,
        [
            locations["fakta"],
            [55.36243178318969, 10.40737547068215],
            locations["home"],
        ]
    ],
    "fakta-home-devi2": [True, 1,
        [
            locations["fakta"],
            [55.36278400458655, 10.404573436222652],
            [55.36478675391273, 10.406346657883642],
            [55.36535525783513, 10.407642473712826],
            locations["home"],
        ]
    ],
    "fakta-home-devi3": [True, 1,
        [
            locations["fakta"],
            [55.3629572473624, 10.403128344570984],
            [55.36487197631148, 10.403793532396445],
            [55.36544515740819, 10.405531603811362],
            locations["home"],
        ]
    ],
    "fakta-home-devi4": [True, 1,
        [
            locations["fakta"],
            [55.36267250064205, 10.397291211737615],
            [55.35988286163377, 10.39727316805758],
            [55.361964853475, 10.403281713508438],
            [55.36625156567692, 10.405861959753098],
            locations["home"],
        ]
    ],
    ####  home --> lidl (5 + 1) -->
    "home-lidl-norm":  [False, 6, [locations["home"], locations["lidl"]]],
    "home-lidl-devi1": [True, 1,
        [
            locations["home"],
            [55.368654525477595, 10.40214269932413],
            [55.3708337247534, 10.402759934790717],
            locations["lidl"],
        ]
    ],
    "home-lidl-devi2": [True, 1,
        [
            locations["home"],
            [55.370331240589394, 10.406725459642242],
            [55.37128103764875, 10.406597871562429],
            locations["lidl"],
        ]
    ],
    "home-lidl-devi3": [True, 1,
        [
            locations["home"],
            [55.3689231075168, 10.398002999198704],
            [55.37116098319547, 10.40152045581184],
            locations["lidl"],
        ]
    ],
    "home-lidl-devi4": [True, 1,
        [
            locations["home"],
            [55.369053472678395, 10.393548828596202],
            [55.37191053461741, 10.395020807722354],
            [55.37123702529128, 10.401443989363726],
            locations["lidl"],
        ]
    ],
    "lidl-home-norm":  [False, 11 + 5, [locations["lidl"], locations["home"]]],
    #### home --> land (5 + 1) --> fakta (6)
    "home-land-norm":  [False, 6, [locations["home"], locations["land"]]],
    "home-land-devi1": [True, 1,
        [
            locations["home"],
            [55.367116410064604, 10.394117856530032],
            locations["land"],
        ]
    ],  
    "land-fakta-norm":  [False, 7, [locations["land"], locations["fakta"]]],
    "land-fakta-dev1":  [True, 1, 
        [
            locations["land"],
            [55.36663486286559, 10.385983240440687],
            [55.3647143257673, 10.386424969047207],
            [55.36403646689263, 10.389494982862532],
            locations["fakta"],
        ]
    ],
    "land-fakta-dev2":  [True, 1, 
        [
            locations["land"],
            [55.36650934041169, 10.390599304378837],
            [55.369057368236, 10.389406637141228],
            [55.369747692355794, 10.393757663915466],
            locations["fakta"],
        ]
    ],
    #####    
    ##### home --> hcl (5) --> lidl (5)
    ##### hcl --> home (1)
    "home-hcl-norm": [False, 6,
        [
            locations["home"],
            [55.36922148996438, 10.393627298339139],
            locations["hcl"],
        ]
    ],
    "hcl-lidl-norm": [False, 5,
        [
            locations["hcl"],
            [55.37408034921968, 10.403112588773388],
            locations["lidl"],
        ]
    ],
    "hcl-home-devi1": [True, 1,
        [
            locations["hcl"],
            [55.37124927486343, 10.401271583186231],
            [55.37015850074101, 10.406448780293063],
            locations["home"],
        ]
    ],
    "home-lidl-dev01": [True, 1,
        [
            locations["home"],
            [55.36920362839035, 10.393375034104885],
            [55.37764100710188, 10.396407510431787],
            locations["lidl"]
        ]
    ],
    "home-lidl-dev02": [True, 1,
        [
            locations["hcl"],
            [55.378163310200705, 10.403420356235491],
            locations["lidl"]
        ]
    ],
    ####
    "home-uni": [False, 5,
        [
            locations["home"],
            locations["uni"],
        ]
    ],
    "uni-home": [False, 5,
        [
            locations["uni"],
            locations["home"],
        ]
    ],
    ####
    "uni-netto": [False, 5,
        [
            locations["uni"],
            locations["netto"],
        ]
    ],
    "uni-lidl": [False, 5,
        [
            locations["netto"],
            [55.37430882819736, 10.419526006255474],
            locations["lidl"],
        ]
    ], 
    "netto-lidl-dev1": [True, 1,
        [
            locations["netto"],
            [55.37430882819736, 10.419526006255474],
            [55.375186680133844, 10.419204141302785],
            locations["lidl"],
        ]
    ],
    "netto-lidl-dev2": [True, 1,
        [
            locations["netto"],
            [55.37768651143561, 10.417936038444099],
            [55.3771347840795, 10.40877162746623],
            locations["lidl"],
        ]
    ],
    "netto-lidl-dev3": [True, 1,
        [
            locations["netto"],
            [55.37566385963514, 10.427472022680385],
            [55.37415489357414, 10.417473458120758],
            locations["lidl"],
        ]
    ],
    "netto-lidl-dev4": [True, 1,
        [
            locations["netto"],
            [55.37397937146365, 10.414463078620756],
            [55.37631649139682, 10.41398410975295],
            [55.37622224973701, 10.40919955677602],            
            locations["lidl"],
        ]
    ],
    ###
    "home-hcl-dev03": [True, 1,
        [
            locations["home"],
            [55.369061123671486, 10.393362959756557],
            [55.3739168769067, 10.387490391658314],
            locations["hcl"],
        ],
    ],
    "fakta-home-dev01": [True, 1,
        [
            locations["fakta"],
            [55.36231061367746, 10.393748256657942],
            [55.361967249170185, 10.403335105790346],
            [55.36614713623522, 10.405777782418603],
            locations["home"],
        ],
    ],
    ###
}

In [222]:
path_counter_anom = 0
path_counter_norm = 0
for profile in profiles:
    if profiles[profile][0] == True:
        path_counter_anom += profiles[profile][1]
    else:
        path_counter_norm += profiles[profile][1]
print("N paths: ", path_counter_anom + path_counter_norm)
print("Norm   : ", path_counter_norm)
print("Anom   : ", path_counter_anom)
print("Ratio  : ", path_counter_anom / (path_counter_anom + path_counter_norm))

N paths:  108
Norm   :  86
Anom   :  22
Ratio  :  0.2037037037037037


In [221]:
patterns_waypoint_list = []

for profile in profiles:
    patterns_waypoint_list.extend(
        generate_pattern(
            profile = profiles[profile],
        )
    )
patterns_waypoint_list

map_ = setup_map(center)

# map_ = plot_datapoint([start_latitude, start_longitude], color = "red", map_ = map_)
# map_ = plot_datapoint([middle_latitude, middle_longitude], color = "green", map_ = map_)
# map_ = plot_datapoint([end_latitude, end_longitude], color = "green", map_ = map_)

trajectories1 = []
for anomaly, n, waypoints in patterns_waypoint_list:
    for _ in range(n):
        random.seed(time.time())
        color = "#" + str(color_hex(num = rand_24_bit()))
        print(waypoints)
        trajectory_generator = TrajectoryGenerator(
            waypoints = waypoints,
            mean_time_delta = mean_time_delta,
            std_time_delta = std_time_delta,
            mean_speed = mean_speed,
            std_speed = std_speed,
            # https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0219890
            # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6638960/
            # np.mean(np.abs(np.random.normal(0, 9, size=1000))) approx 7
            std_datapoint = 7.5,
        )
        map_ = plot_trajectory(
            trajectory_generator.to_latlon(),
            color = color,
            map_ = map_,
        )
        trajectories1.append((anomaly, trajectory_generator))
map_

[<TrajGen.WayPoint object at 0x7f31414793a0>, <TrajGen.WayPoint object at 0x7f31414795b0>]
[<TrajGen.WayPoint object at 0x7f31414793a0>, <TrajGen.WayPoint object at 0x7f31414795b0>]
[<TrajGen.WayPoint object at 0x7f31414793a0>, <TrajGen.WayPoint object at 0x7f31414795b0>]
[<TrajGen.WayPoint object at 0x7f31414793a0>, <TrajGen.WayPoint object at 0x7f31414795b0>]
[<TrajGen.WayPoint object at 0x7f31414793a0>, <TrajGen.WayPoint object at 0x7f31414795b0>]
[<TrajGen.WayPoint object at 0x7f31414793a0>, <TrajGen.WayPoint object at 0x7f31414795b0>]
[<TrajGen.WayPoint object at 0x7f3141479430>, <TrajGen.WayPoint object at 0x7f3141479610>, <TrajGen.WayPoint object at 0x7f3141479490>]
[<TrajGen.WayPoint object at 0x7f3141479430>, <TrajGen.WayPoint object at 0x7f3141479610>, <TrajGen.WayPoint object at 0x7f3141479490>]
[<TrajGen.WayPoint object at 0x7f3141479430>, <TrajGen.WayPoint object at 0x7f3141479610>, <TrajGen.WayPoint object at 0x7f3141479490>]
[<TrajGen.WayPoint object at 0x7f3141479430>, 

[<TrajGen.WayPoint object at 0x7f3141446fa0>, <TrajGen.WayPoint object at 0x7f31414a40a0>, <TrajGen.WayPoint object at 0x7f31414a4040>]
[<TrajGen.WayPoint object at 0x7f3141446fa0>, <TrajGen.WayPoint object at 0x7f31414a40a0>, <TrajGen.WayPoint object at 0x7f31414a4040>]
[<TrajGen.WayPoint object at 0x7f3141446fa0>, <TrajGen.WayPoint object at 0x7f31414a40a0>, <TrajGen.WayPoint object at 0x7f31414a4040>]
[<TrajGen.WayPoint object at 0x7f3141446fa0>, <TrajGen.WayPoint object at 0x7f31414a40a0>, <TrajGen.WayPoint object at 0x7f31414a4040>]
[<TrajGen.WayPoint object at 0x7f3141446fa0>, <TrajGen.WayPoint object at 0x7f31414a40a0>, <TrajGen.WayPoint object at 0x7f31414a4040>]
[<TrajGen.WayPoint object at 0x7f3141446fa0>, <TrajGen.WayPoint object at 0x7f31414a40a0>, <TrajGen.WayPoint object at 0x7f31414a4040>]
[<TrajGen.WayPoint object at 0x7f31414a4100>, <TrajGen.WayPoint object at 0x7f31414a41c0>, <TrajGen.WayPoint object at 0x7f31414a4130>]
[<TrajGen.WayPoint object at 0x7f31414a4100>, <T

In [191]:
def generate_trajectories(
    waypoints: List,
    start_datetime: datetime,
    gap: timedelta = timedelta(days = 1.),
    *args, **kwargs,
    ) -> TrajectoryGeneratorCollection: 
    trajectories = []
    for anomaly, n, waypoints in waypoints:
        for _ in range(n):
            random.seed(time.time())
            color = "#" + str(color_hex(num = rand_24_bit()))
            trajectory_generator = TrajectoryGenerator(waypoints = waypoints, *args, **kwargs)
            trajectories.append((anomaly, trajectory_generator))
    return TrajectoryGeneratorCollection(
        trajectory_generators = trajectories,
        start_datetime = start_datetime,
        gap = gap,
    ) 

In [ ]:
def generate_batches_deviating(n_batches, seeds):
    batches = []
    for seed in seeds:
        waypoints = generate_deviating_patterns(
            seed = seed,
            n_normal = 10,
            n_deviating = 5, 
        ) 
        kwargs = {
            "mean_time_delta": mean_time_delta,
            "std_time_delta": std_time_delta,
            "mean_speed": mean_speed,
            "std_speed": std_speed,
            "std_datapoint": 9, # 9
        }
        tgc = generate_trajectories(
            waypoints = waypoints, 
            start_datetime = datetime.now(),
            **kwargs,
        )
        main_df = tgc.to_dataframe(anom_start = True)
        batches.append(main_df.to_json(orient = "split"))
    return batches


# def read_batches(data_in):
# #     data_in = None; #df = 
# #     with open(json_file, "r") as f:
# #         data_in = json.loads(f.read())
#     batches = {}; 
#     for i, batch in enumerate(data_in): # Get each batch of trajectories
#         df_batch = pd.read_json(batch, orient = "split")
#         _data = []
#         for _, row in df_batch.iterrows():
#             df_traj = pd.read_json(row["df"], orient = "split")
#             _data.append(df_traj)
#         df_batch["df"] = _data
#         batches[i] = df_batch 
#     return batches

    
# def generate_batches_looping(n_batches):
#     batches = []
#     seeds = [i for i in range(12, 12 + n_batches + 1)]
#     for seed in seeds:
#         waypoints = generate_looping_patterns(
#             seed = seed,
#             n_normal = 10,
#             n_looping = 5, 
#         ) 
#         kwargs = {
#             "mean_time_delta": mean_time_delta,
#             "std_time_delta": std_time_delta,
#             "mean_speed": mean_speed,
#             "std_speed": std_speed,
#             "std_datapoint": 14,
#         }
#         tgc = generate_trajectories(
#             waypoints = waypoints, 
#             start_datetime = datetime.now(),
#             **kwargs,
#         )
#         main_df = tgc.to_dataframe()
#         batches.append(main_df.to_json(orient = "split"))
#     return batches

# batches0 = generate_batches_deviating(n_batches = 1)

In [ ]:
start_seed = 13
seeds = [2, 3, 5, 6, 8] # [i for i in range(start_seed, start_seed + n_batches + 1)]
n_batches = len(seeds)
batches0 = generate_batches_deviating(n_batches = n_batches, seeds = seeds)
with open("main_data_deviating_large.json", "w") as f:
    f.write(json.dumps(batches0))